### Εργασία του Φοιτητή Μυλωνά Αθανασίου ΑΜ: e21118


In [1]:
# Εισαγωγή βιβλιοθηκών
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk

## Εισάγουμε τα δεδομένα για πρώτη φορά


In [2]:
#Φορτώνουμε τα δεδομένα
rawData = pd.read_csv('/home/th4nos/Projects/DataScience/CourseAssignment/Data/data.csv')

# Φτιάχνω ένα αντίγραφο των δεδομένων για λόγους ασφαλείας
data = rawData.copy()
#Εμφανίζουμε τα πρώτα 5 δείγματα
data.head()

,TripId,StartTime,EndTime,StartStationId,EndStationId
0,2616540,2023-02-01 00:28:04,2023-02-01 00:45:18,1,0
1,2616644,2023-02-01 00:34:08,2023-02-01 00:45:49,1,0
2,2616730,2023-02-01 00:39:18,2023-02-01 00:52:06,1,0
3,2617099,2023-02-01 00:59:02,2023-02-01 01:08:31,0,1
4,2617103,2023-02-01 00:59:35,2023-02-01 01:08:19,0,1


### Σαν πρώτη όψη

Το dataset περιλαμβάνει πέντε στήλες: το μοναδικό αναγνωριστικό κάθε διαδρομής (TripId), την ώρα εκκίνησης (StartTime), την ώρα επιστροφής (EndTime), το αναγνωριστικό του σταθμού αφετηρίας (StartStationId) και το αναγνωριστικό του σταθμού προορισμού (EndStationId). Κάθε εγγραφή καταγράφει μια διαδρομή ποδηλάτου με πληροφορίες για τον χρόνο και τους σταθμούς εκκίνησης και προορισμού.


#### Προετοιμασία

Στην συνέχεια θα κάνουμε έναν πρώτο έλεγχο των δεδομένων για να δούμε τι περιέχουν και πως είναι οργανωμένα.


In [3]:
#Εμφανίζουμε τις πληροφορίες των δεδομένων
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8636 entries, 0 to 8635
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   TripId          8636 non-null   int64 
 1   StartTime       8636 non-null   object
 2   EndTime         8636 non-null   object
 3   StartStationId  8636 non-null   int64 
 4   EndStationId    8636 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 337.5+ KB


Με την εμφάνισει των πληροφοριών σχετικά για τι τύπο δεδομένων περιέχει κάθε στήλη, μπορούμε να καταλάβουμε τι είδους επεξεργασία θα χρειαστεί να κάνουμε. Στην συνέχεια θα ελέγξουμε αν υπάρχουν κενές τιμές στο dataset όπως επίσης και αν υπάρχουν διπλές εγγραφές.


In [4]:
# Ελέγχουμε για τυχόν απουσιάζουσες τιμές
data.isnull().sum()

TripId            0
StartTime         0
EndTime           0
StartStationId    0
EndStationId      0
dtype: int64

In [5]:
# Ελέγχουμε για τυχόν διπλές τιμές στο TripId
data.duplicated(subset=['TripId']).sum()

0

In [6]:
# Επίσης ελέγχουμε για το άμα το StartStationId και EndStationId έχουν και τα δύο ίδιες τιμές
dupleCheck = (data['StartStationId'] == data['EndStationId']).all()
dupleCheck

False

Τελικά δεν υπάρχουν κενές τιμές ή διπλές εγγραφές στο dataset.


#### Σαν μια πρώτη αλλάγη θα μετατρέψουμε τις στήλες StartTime και EndTime σε datetime αντικείμενα από object για να μπορούμε να κάνουμε εύκολα υπολογισμούς μεταξύ τους.


In [7]:
# Mετατρέπουμε τις στήλες 'StartTime' και 'EndTime' σε datetime
data['StartTime'] = pd.to_datetime(data['StartTime'])
data['EndTime'] = pd.to_datetime(data['EndTime'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8636 entries, 0 to 8635
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   TripId          8636 non-null   int64         
 1   StartTime       8636 non-null   datetime64[ns]
 2   EndTime         8636 non-null   datetime64[ns]
 3   StartStationId  8636 non-null   int64         
 4   EndStationId    8636 non-null   int64         
dtypes: datetime64[ns](2), int64(3)
memory usage: 337.5 KB


### Στην εκφώνηση της άσκησης μας εξηγεί πως:

**"Η τιμή 1 σε κάποια από τις στήλες StartStationId ή EndStationId υποδηλώνει το σταθμό που μας ενδιαφέρει. Αντίθετα, όπου αναγράφεται η τιμή 0 σε κάποια από τις στήλες αυτές, αυτή υποδηλώνει κάποιον άλλον σταθμό (̸ = 1) που μας είναι αδιάφορος."**

Επομένως θα πρέπει να φιλτράρουμε τα δεδομένα μας ώστε να περιέχουν μόνο τις διαδρομές που μας ενδιαφέρουν.


In [8]:
# Φιλτράρουμε τα δεδομένα με βάση το άμα η StartStationId ή η EndStationId είναι 1
filteredData = data[(data['StartStationId'] == 1) | (data['EndStationId'] == 1)]
filteredData

,TripId,StartTime,EndTime,StartStationId,EndStationId
0,2616540,2023-02-01 00:28:04,2023-02-01 00:45:18,1,0
1,2616644,2023-02-01 00:34:08,2023-02-01 00:45:49,1,0
2,2616730,2023-02-01 00:39:18,2023-02-01 00:52:06,1,0
3,2617099,2023-02-01 00:59:02,2023-02-01 01:08:31,0,1
4,2617103,2023-02-01 00:59:35,2023-02-01 01:08:19,0,1
...,...,...,...,...,...
8631,3988772,2023-02-28 23:29:08,2023-02-28 23:33:32,1,0
8632,3988863,2023-02-28 23:33:01,2023-02-28 23:36:30,0,1
8633,3988901,2023-02-28 23:35:32,2023-02-28 23:40:22,1,0
8634,3988930,2023-02-28 23:36:58,2023-02-28 23:45:17,0,1


#### Για να κάνουμε ανάλυση χρονοσειρών θα χρειαστεί να επιλέξουμε ένα χρονικό διάστημα. Στην συγκεκριμένη περίπτωση θα διαλέξουμε το παράδειγμα που δίνει και στην εκφώνηση που είναι οι **"T = 2 ώρες"**. Αφού ομαδοποιήσουμε τις εκκινήσεις και τις επιστροφές σε κάθε διάστημα **T**, υπολογίζουμε τη ζήτηση ως τη διαφορά μεταξύ των ποδηλάτων που επέστρεψαν και αυτών που παραδόθηκαν εντός του διαστήματος **T**. Με αυτόν τον τρόπο, δημιουργούμε μια χρονοσειρά ζήτησης ποδηλάτων.


In [9]:
# Ορίζουμε το χρονικό διάστημα που θέλουμε να εξετάσουμε
T = '2h'  # Ορισμός του χρονικού διαστήματος σε 2 ώρες

# Δημιουργία της χρονοσειράς για την εκκίνηση από τον ενδιαφερόμενο σταθμό
startSeries = filteredData[filteredData['StartStationId'] == 1].set_index('StartTime').resample(T).count()['TripId']

# Δημιουργία της χρονοσειράς για τον τερματισμό στον ενδιαφερόμενο σταθμό
endSeries = filteredData[filteredData['EndStationId'] == 1].set_index('EndTime').resample(T).count()['TripId']

# Υπολογισμός ζήτησης ποδηλάτων στον ενδιαφερόμενο σταθμό & Δημιουργία της χρονοσειράς
demandSeries = endSeries.sub(startSeries, fill_value=0)

# Εκτύπωση της χρονοσειράς
demandSeries = pd.DataFrame({'Time': demandSeries.index, 'Demand': demandSeries.values})
demandSeries

,Time,Demand
0,2023-02-01 00:00:00,-1
1,2023-02-01 02:00:00,0
2,2023-02-01 04:00:00,1
3,2023-02-01 06:00:00,-10
4,2023-02-01 08:00:00,-7
...,...,...
331,2023-02-28 14:00:00,4
332,2023-02-28 16:00:00,8
333,2023-02-28 18:00:00,6
334,2023-02-28 20:00:00,4


### Χωρισμός των δεδομένων σε train και test set

Θα χωρίσουμε τα δεδομένα σε δύο μέρη: το πρώτο 70% για εκπαίδευση και το υπόλοιπο 30% για έλεγχο.


In [10]:
# Ορίζουμε το ποσοστό των δεδομένων που θα χρησιμοποιήσουμε για το training
trainSize = 0.7

# Υπολογίζουμε τον αριθμό των παρατηρήσεων για εκπαίδευση
trainSize = int(len(demandSeries) * trainSize)

# Διαχωρίζουμε τα δεδομένα σε train και test
trainData = demandSeries[:trainSize]
testData = demandSeries[trainSize:]

# Εφαρμογή μεθόδων πρόβλεψης χρονοσειρών


### Εφαρμογή Εκθετικής Εξομάλυνσης (Exponential Smoothing)

Η Εκθετική Εξομάλυνση αποτελεί μια τεχνική πρόβλεψης χρονοσειρών που δίνει μεγαλύτερη σημασία στις πιο πρόσφατες παρατηρήσεις, με μείωση της σπουδαιότητας για τις παλαιότερές τους. Αυτό επιτυγχάνεται μέσω της χρήση ενός συντελέσμα (a) που κυμαίνεται από 0 έως 1.


In [11]:
# Εισαγωγή της βιβλιοθήκης statsmodels
from statsmodels.tsa.holtwinters import ExponentialSmoothing
# Εισαγωγή των μετρικών MAE και RMSE με σκοπό να δούμε τα στατιστικά μέτρα απόδοσης των προβλέψεων
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [12]:
# Δημιουργία αντιγράφου του test set για να αποφύγουμε το SettingWithCopyWarning
testDataCopy = testData.copy()

# Εκπαίδευση του μοντέλου
modelExp = ExponentialSmoothing(trainData['Demand'], seasonal='add', seasonal_periods=12).fit()

# Πρόβλεψη με το μοντέλο
predExp = modelExp.forecast(len(testDataCopy))

# Oρισμός των προβλέψεων στο test set χρησιμοποιώντας .loc
testDataCopy.loc[:, 'PredictionsEXP'] = predExp.values

# Ανάθεση των προβλέψεων στο αντίγραφο του test set
expPredictions = testDataCopy.copy()

# Εκτύπωση των προβλέψεων
expPredictions

,Time,Demand,PredictionsEXP
235,2023-02-20 14:00:00,13,4.526310
236,2023-02-20 16:00:00,-3,3.210517
237,2023-02-20 18:00:00,9,0.578943
238,2023-02-20 20:00:00,4,2.947364
239,2023-02-20 22:00:00,5,5.210524
...,...,...,...
331,2023-02-28 14:00:00,4,4.526310
332,2023-02-28 16:00:00,8,3.210517
333,2023-02-28 18:00:00,6,0.578943
334,2023-02-28 20:00:00,4,2.947364


In [13]:
# Υπολογισμός των μετρικών MAE και RMSE
maeEXP = mean_absolute_error(testDataCopy['Demand'], testDataCopy['PredictionsEXP'])
rmseEXP = np.sqrt(mean_squared_error(testDataCopy['Demand'], testDataCopy['PredictionsEXP']))

print(f'Mean Absolute Error (MAE) of EXP: {maeEXP}')
print(f'Root Mean Square Error (RMSE) of EXP: {rmseEXP}')

Mean Absolute Error (MAE) of EXP: 3.1966655095750025
Root Mean Square Error (RMSE) of EXP: 4.200916737860467


## ARIMA (AutoRegressive Integrated Moving Average)

Το ARIMA (Autoregressive Integrated Moving Average) αποτελεί μια αξιόπιστη μέθοδο πρόβλεψης χρονοσειρών, λαμβάνοντας υπόψη αυτοπαλινδρομήσεις, διαφοροποιήσεις και κινούμενους μέσους όρους. Η μέθοδος χαρακτηρίζεται από τους παράγοντες p, d και q, οι οποίοι καθορίζουν τις τάξεις των αντίστοιχων συνιστωσών.

Η ευελιξία και η ακρίβεια του ARIMA, ιδανικά για βραχυπρόθεσμες προβλέψεις, το καθιστούν δημοφιλές εργαλείο σε διάφορες εφαρμογές ανάλυσης δεδομένων.


In [14]:
# Εισαγωγή της βιβλιοθήκης του ARIMA
from statsmodels.tsa.arima.model import ARIMA

# Εισαγωγή της βιβλιοθήκης του auto_arima για την αυτόματη επιλογή των παραμέτρων
from pmdarima.arima import auto_arima

In [15]:
# Αυτόματη εύρεση βέλτιστων παραμέτρων ARIMA
modelARIMA = auto_arima(trainData['Demand'], seasonal=True, trace=True, m=12)

# Εκπαίδευση του μοντέλου ARIMA με τις βέλτιστες παραμέτρους
modelARIMA.fit(trainData['Demand'])

# Πρόβλεψη για το test set
predARIMA = modelARIMA.predict(len(testDataCopy))

# Προσθήκη των προβλέψεων στο test set
testDataCopy['PredictionsARIMA'] = predARIMA

# Ανάθεση των προβλέψεων στο αντίγραφο του test set
arimaPredictions = testDataCopy.copy()

# Εκτύπωση των προβλέψεων
arimaPredictions

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=0.69 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=1538.316, Time=0.01 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=1508.148, Time=0.09 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=1514.361, Time=0.23 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=1536.337, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=1535.885, Time=0.04 sec
 ARIMA(1,0,0)(2,0,0)[12] intercept   : AIC=1505.229, Time=0.50 sec
 ARIMA(1,0,0)(2,0,1)[12] intercept   : AIC=inf, Time=1.09 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=0.97 sec
 ARIMA(0,0,0)(2,0,0)[12] intercept   : AIC=1504.369, Time=0.30 sec
 ARIMA(0,0,0)(1,0,0)[12] intercept   : AIC=1506.517, Time=0.09 sec
 ARIMA(0,0,0)(2,0,1)[12] intercept   : AIC=inf, Time=1.30 sec
 ARIMA(0,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=0.44 sec
 ARIMA(0,0,1)(2,0,0)[12] intercept   : AIC=1505.189, Time=0.33 sec
 ARIMA(1,0,1)(2,0,0)[12] intercept   : AIC=i

,Time,Demand,PredictionsEXP,PredictionsARIMA
235,2023-02-20 14:00:00,13,4.526310,0.273129
236,2023-02-20 16:00:00,-3,3.210517,1.132093
237,2023-02-20 18:00:00,9,0.578943,0.880710
238,2023-02-20 20:00:00,4,2.947364,0.039578
239,2023-02-20 22:00:00,5,5.210524,0.625413
...,...,...,...,...
331,2023-02-28 14:00:00,4,4.526310,0.001775
332,2023-02-28 16:00:00,8,3.210517,0.008945
333,2023-02-28 18:00:00,6,0.578943,0.013665
334,2023-02-28 20:00:00,4,2.947364,0.001846


In [16]:
# Υπολογισμός μετρικών αξιολόγησης
maeARIMA = mean_absolute_error(testDataCopy['Demand'], testDataCopy['PredictionsARIMA'])
rmseARIMA = np.sqrt(mean_squared_error(testDataCopy['Demand'], testDataCopy['PredictionsARIMA']))

# Εκτύπωση των αποτελεσμάτων
print(f"Mean Absolute Error (MAE) of ARIMA: {maeARIMA}")
print(f"Root Mean Square Error (RMSE) of ARIMA: {rmseARIMA}")

Mean Absolute Error (MAE) of ARIMA: 4.273538989288767
Root Mean Square Error (RMSE) of ARIMA: 5.662298077734


## Εξέταση των Aποτελεσμάτων της Προβολής

Κατά τη σύγκριση των τεχνικών πρόβλεψης που αξιολογήσαμε, ανακαλύπτουμε ότι η Εκθετική Εξομάλυνση αποδίδει με μεγαλύτερη ακρίβεια από την ARIMA.

Ας εξετάσουμε τα αποτελέσματα με περισσότερες λεπτομέρειες:
- Μέσο απόλυτο σφάλμα (MAE): Περίπου 3,20 για την Εκθετική Εξομάλυνση- Περίπου 4,27 για την ARIMA

Η μέση διαφορά μεταξύ των αναμενόμενων και των πραγματικών τιμών καθορίζεται από το MAE. Η πρόβλεψη είναι πιο ακριβής όσο χαμηλότερη είναι η τιμή του MAE. Η Εκθετική Εξομάλυνση, στην προκειμένη περίπτωση, εμφανίζει αισθητά χαμηλότερο MAE, υποδεικνύοντας καλύτερη προσαρμογή στις πραγματικές τιμές.
- RMSE, ή μέσο τετραγωνικό σφάλμα:
    - Εκθετική Εξομάλυνση: περίπου 4,20- ARIMA: περίπου 5,66.


Η τυπική απόκλιση μεταξύ των αναμενόμενων και των πραγματικών τιμών μετράται από το RMSE. Για άλλη μια φορά, μια χαμηλότερη τιμή RMSE της μεθόδου Εκθετική Εξομάλυνσης σημαίνει χαμηλότερη διακύμανση στις προβλέψεις, η οποία μεταφράζεται σε υψηλότερη ακρίβεια.

Συνοψίζοντας:

Σε σύγκριση με την ARIMA χρησιμοποιώντας τις παραμέτρους που εξετάσαμε, η Εκθετική Εξομάλυνση αποδεικνύεται η πιο επιτυχημένη τεχνική πρόβλεψης για τα δεδομένα σας, σύμφωνα με τα στατιστικά στοιχεία MAE και RMSE.